# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 20 2022 10:50AM,249067,19,8671744,"Graystone, LLC",Released
1,Oct 20 2022 10:50AM,249067,19,8671746,"Graystone, LLC",Released
2,Oct 20 2022 10:34AM,249066,10,8671743,"Coronado Aesthetics, LLC",Released
3,Oct 20 2022 10:17AM,249064,10,8667352,Eywa Pharma Inc.,Released
4,Oct 20 2022 10:17AM,249064,10,8667353,Eywa Pharma Inc.,Released
5,Oct 20 2022 10:17AM,249064,10,8667355,Eywa Pharma Inc.,Released
6,Oct 20 2022 10:17AM,249064,10,8667356,Eywa Pharma Inc.,Released
7,Oct 20 2022 10:17AM,249064,10,8667357,Eywa Pharma Inc.,Released
8,Oct 20 2022 10:17AM,249064,10,8667360,Eywa Pharma Inc.,Released
9,Oct 20 2022 10:17AM,249064,10,8667361,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,249063,Released,1
28,249064,Released,14
29,249065,Released,1
30,249066,Released,1
31,249067,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249063,NaN,NaN,1.0
249064,NaN,NaN,14.0
249065,NaN,NaN,1.0
249066,NaN,NaN,1.0
249067,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248896,0.0,0.0,1.0
248961,0.0,0.0,1.0
248979,0.0,0.0,2.0
248983,0.0,0.0,1.0
248984,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248896,0,0,1
248961,0,0,1
248979,0,0,2
248983,0,0,1
248984,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248896,0,0,1
1,248961,0,0,1
2,248979,0,0,2
3,248983,0,0,1
4,248984,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248896,,,1
1,248961,,,1
2,248979,,,2
3,248983,,,1
4,248984,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 20 2022 10:50AM,249067,19,"Graystone, LLC"
2,Oct 20 2022 10:34AM,249066,10,"Coronado Aesthetics, LLC"
3,Oct 20 2022 10:17AM,249064,10,Eywa Pharma Inc.
17,Oct 20 2022 10:13AM,249065,10,Eywa Pharma Inc.
18,Oct 20 2022 9:36AM,249063,21,"NBTY Global, Inc."
19,Oct 20 2022 9:31AM,249061,10,ISDIN Corporation
23,Oct 20 2022 8:55AM,249054,10,ISDIN Corporation
43,Oct 20 2022 8:46AM,249053,19,"AdvaGen Pharma, Ltd"
94,Oct 20 2022 8:40AM,249051,10,Eywa Pharma Inc.
95,Oct 20 2022 8:06AM,249048,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 20 2022 10:50AM,249067,19,"Graystone, LLC",,,2
1,Oct 20 2022 10:34AM,249066,10,"Coronado Aesthetics, LLC",,,1
2,Oct 20 2022 10:17AM,249064,10,Eywa Pharma Inc.,,,14
3,Oct 20 2022 10:13AM,249065,10,Eywa Pharma Inc.,,,1
4,Oct 20 2022 9:36AM,249063,21,"NBTY Global, Inc.",,,1
5,Oct 20 2022 9:31AM,249061,10,ISDIN Corporation,,,4
6,Oct 20 2022 8:55AM,249054,10,ISDIN Corporation,,3,17
7,Oct 20 2022 8:46AM,249053,19,"AdvaGen Pharma, Ltd",,41,10
8,Oct 20 2022 8:40AM,249051,10,Eywa Pharma Inc.,,,1
9,Oct 20 2022 8:06AM,249048,10,Emerginnova,,5,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 20 2022 10:50AM,249067,19,"Graystone, LLC",2,,
1,Oct 20 2022 10:34AM,249066,10,"Coronado Aesthetics, LLC",1,,
2,Oct 20 2022 10:17AM,249064,10,Eywa Pharma Inc.,14,,
3,Oct 20 2022 10:13AM,249065,10,Eywa Pharma Inc.,1,,
4,Oct 20 2022 9:36AM,249063,21,"NBTY Global, Inc.",1,,
5,Oct 20 2022 9:31AM,249061,10,ISDIN Corporation,4,,
6,Oct 20 2022 8:55AM,249054,10,ISDIN Corporation,17,3,
7,Oct 20 2022 8:46AM,249053,19,"AdvaGen Pharma, Ltd",10,41,
8,Oct 20 2022 8:40AM,249051,10,Eywa Pharma Inc.,1,,
9,Oct 20 2022 8:06AM,249048,10,Emerginnova,1,5,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 20 2022 10:50AM,249067,19,"Graystone, LLC",2,,
1,Oct 20 2022 10:34AM,249066,10,"Coronado Aesthetics, LLC",1,,
2,Oct 20 2022 10:17AM,249064,10,Eywa Pharma Inc.,14,,
3,Oct 20 2022 10:13AM,249065,10,Eywa Pharma Inc.,1,,
4,Oct 20 2022 9:36AM,249063,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 20 2022 10:50AM,249067,19,"Graystone, LLC",2.0,NaN,NaN
1,Oct 20 2022 10:34AM,249066,10,"Coronado Aesthetics, LLC",1.0,NaN,NaN
2,Oct 20 2022 10:17AM,249064,10,Eywa Pharma Inc.,14.0,NaN,NaN
3,Oct 20 2022 10:13AM,249065,10,Eywa Pharma Inc.,1.0,NaN,NaN
4,Oct 20 2022 9:36AM,249063,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 20 2022 10:50AM,249067,19,"Graystone, LLC",2.0,0.0,0.0
1,Oct 20 2022 10:34AM,249066,10,"Coronado Aesthetics, LLC",1.0,0.0,0.0
2,Oct 20 2022 10:17AM,249064,10,Eywa Pharma Inc.,14.0,0.0,0.0
3,Oct 20 2022 10:13AM,249065,10,Eywa Pharma Inc.,1.0,0.0,0.0
4,Oct 20 2022 9:36AM,249063,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3486372,47.0,37.0,2.0
16,249043,0.0,1.0,0.0
19,1245141,22.0,69.0,2.0
20,248961,1.0,0.0,0.0
21,497959,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3486372,47.0,37.0,2.0
1,16,249043,0.0,1.0,0.0
2,19,1245141,22.0,69.0,2.0
3,20,248961,1.0,0.0,0.0
4,21,497959,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,47.0,37.0,2.0
1,16,0.0,1.0,0.0
2,19,22.0,69.0,2.0
3,20,1.0,0.0,0.0
4,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,47.0
1,16,Released,0.0
2,19,Released,22.0
3,20,Released,1.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21
Status,,,,,
Completed,2.0,0.0,2.0,0.0,0.0
Executing,37.0,1.0,69.0,0.0,0.0
Released,47.0,0.0,22.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21
0,Completed,2.0,0.0,2.0,0.0,0.0
1,Executing,37.0,1.0,69.0,0.0,0.0
2,Released,47.0,0.0,22.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21
0,Completed,2.0,0.0,2.0,0.0,0.0
1,Executing,37.0,1.0,69.0,0.0,0.0
2,Released,47.0,0.0,22.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()